In [1]:
import os
from os import listdir
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import re
import pathlib

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 26.9 MB/s eta 0:00:00


In [ ]:
# Define your dataset directory
Agriculture_crop = '/content/drive/MyDrive/dataset'

# Define types
types = [
    "1", "2", "3", "4", "5"
]

# Initialize lists to hold images and their labels
CrownPreparationimages = []
CrownPreparationlabels = []




target_size = (224, 224)  # Desired size to resize images

for i, PreparationType in enumerate(types, start=1):
    CrownPreparationFolder = os.path.join(Agriculture_crop, str(i))
    for root, dirs, files in os.walk(CrownPreparationFolder):
        for file_name in files:
            img_path = os.path.join(root, file_name)
            _, file_extension = os.path.splitext(img_path)
            if file_extension.lower() == '.jpeg' or file_extension.lower() == '.jpg':
            #for add photo output folder
            #if file_extension.lower() == '.jpg':
                img = Image.open(img_path)
            elif file_extension.lower() == '.heic':

                try:
                    heif_file = pyheif.read(img_path)
                    img = Image.frombytes(
                        heif_file.mode,
                        heif_file.size,
                        heif_file.data,
                        "raw",
                        heif_file.mode,
                        heif_file.stride,
                    )
                except Exception as e:
                    print(f"Error opening {img_path}: {e}")
                    continue
            else:
                print(f"Unsupported file format: {img_path}")
                continue

            # Resize image to target size
            img = img.resize(target_size)

            # Append image and label
            CrownPreparationimages.append(np.array(img))
            CrownPreparationlabels.append(PreparationType)

CrownPreparationimages = np.array(CrownPreparationimages)
CrownPreparationlabels = np.array(CrownPreparationlabels)

In [33]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

print("Number of images loaded:", len(CrownPreparationimages))
print("Number of labels:", len(CrownPreparationlabels))


Number of images loaded: 828
Number of labels: 828


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense


# Split the dataset into training and testing
train_images, test_images, train_labels, test_labels = train_test_split(
    CrownPreparationimages, CrownPreparationlabels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(label_binarizer.classes_), activation="softmax"))
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping
history = model.fit(train_images, train_labels, epochs=100,
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/100
21/21 [==============================] - 204s 10s/step - loss: 82.2844 - accuracy: 0.2130 - val_loss: 1.6088 - val_accuracy: 0.1928
Epoch 2/100
21/21 [==============================] - 210s 10s/step - loss: 1.6086 - accuracy: 0.1888 - val_loss: 1.6093 - val_accuracy: 0.2229
Epoch 3/100
15/21 [====================>.........] - ETA: 50s - loss: 1.6075 - accuracy: 0.2083

In [ ]:
# IMAGE SIZE = 128,128
#--------- DATASET = 201
#1 32,64,128 -> 29
#2 32,32,64,64 WITH DROPOT(0.25)-> 24
#3 64,64,128,512 -> 31.70

#--------- DATASET = 804
#4 64,64,128,512 WITH DROPOT(0.25)-> 19
#5 64,64,128,128 WITH DROPOT(0.25)-> 16

# IMAGE SIZE = 224,224
#--------- DATASET = 804
#6 64,64,128,128 WITH DROPOT(0.25)-> 16
#7 32,64,128 -> 16
#new code
#8 32,32,64,64 WITH DROPOT(0.25)-> 42.85
#9 32,32,64,64,128,128 WITH DROPOT(0.25)->19.75

In [ ]:
import tensorflow as tf

# Assuming 'model' is your trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from collections import Counter

# Define your preprocessing function
def preprocess_image(image_path, target_size=(128, 128)):
    # Load image and resize
    img = load_img(image_path, target_size=target_size)
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    return img_array

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/model.tflite")
interpreter.allocate_tensors()

# Load test images (assuming you have a list of file paths)
test_image_paths = [
    '/content/drive/MyDrive/five_views/2/IMAGE 1445-11-06 02:09:24.jpg',
    '/content/drive/MyDrive/five_views/2/IMAGE 1445-11-06 02:09:28.jpg',
    '/content/drive/MyDrive/five_views/2/IMAGE 1445-11-06 02:09:31.jpg',
    '/content/drive/MyDrive/five_views/2/IMAGE 1445-11-06 02:09:34.jpg',
    '/content/drive/MyDrive/five_views/2/IMAGE 1445-11-06 02:09:37.jpg'
]
types = [
    "Finish line position", "Finish line continuity", "Finish line irregularity", "taper",
    "line angle roundness", "axial wall undercut", "amount of occlusal reduction",
    "Over reduction", "Ideal"
]

# Preprocess test images
test_images = [preprocess_image(image_path) for image_path in test_image_paths]
test_images = np.array(test_images)

# Number of top labels to retrieve for each image
N = 3  # Adjust as needed

# Prepare input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference for each test image
predicted_labels_all = []

for image in test_images:
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(image, axis=0))

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Sort predictions by probability and get top N labels
    top_n_indices = np.argsort(output_data[0])[::-1][:N]  # Adjust N as needed
    top_n_labels = [(index, output_data[0][index]) for index in top_n_indices]

    # Append top N labels to the list
    predicted_labels_all.extend(top_n_indices)

# Count the occurrences of each label
label_counts = Counter(predicted_labels_all)

# Filter labels that appeared at least in two images
final_labels = [label for label, count in label_counts.items() if count >= 2 and types[label] != "Ideal"]

# Print the final predicted labels
print("Final Predicted Labels:")
for label in final_labels:
    if label < len(types):
        print(f"- {types[label]}")
        if types[label] == "Finish line position":
            print("Ensure that the finish line is not more than 1.5mm above the gumline and no more than 1.5mm below the gumline.")
        elif types[label] == "taper":
            print("Be sure the taper is not more than 12mm.")
        elif types[label] == "amount of occlusal reduction":
            print("Ensure that the occlusal reduction is not less than 3mm.")
        else:
            print("You have to re-curve the tooth again")
    else:
        print("Excellent work")


Final Predicted Labels:
- amount of occlusal reduction
Ensure that the occlusal reduction is not less than 3mm.
- Finish line irregularity
You have to re-curve the tooth again


In [ ]:
# add one more layer
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),

    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_binarizer.classes_), activation='softmax')
])

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
# Predict the classes of the test images
test_predictions = model.predict(test_images)
test_predictions_classes = np.argmax(test_predictions, axis=1)
test_true_classes = np.argmax(test_labels, axis=1)  # Convert one-hot encoded test labels back to class indices for comparison
# Generate the confusion matrix
cm = confusion_matrix(test_true_classes, test_predictions_classes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


NameError: name 'cm' is not defined

<Figure size 1000x800 with 0 Axes>